<a href="https://colab.research.google.com/github/chrismartel/ECSE415-Project-G7/blob/main/classification/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Download Vehicle Dataset

In [ ]:
!wget -O output https://mcgill-my.sharepoint.com/:u:/g/personal/raghav_mehta_mail_mcgill_ca/EVEvhY9_jyVEk2uSZ8wZhFYBQ58C57I7ZB55jBocKwB5Jg?download=1
!mv output dataset.zip
!unzip dataset.zip
!rm dataset.zip

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import cv2 as cv

def parse(filepath):
  '''
      Parse gt.txt with format
        <frame>, <id>, <type>, <truncated>, <occluded>, <alpha>, <bb_left>, <bb_top>, <bb_right>, <bb_bottom>, <3D_H>, <3D_W>, <3D_L>, <x>, <y>, <z>, <ry>
      Return dict as:
        <type> = "Car", "Van", "Truck", "Pedastrian", "Person_sitting", "Cyclist", "Tram", "Misc", "DontCare"
        key: frame
        value: list - <id>, <bb_left>, <bb_top>, <bb_right>, <bb_bottom>, <is_vehicle>
      Feel free to edit your structure as needed!
  '''

  used_type = ["Car", "Van", "Truck", "Tram"]

  lines = open(filepath, "r").readlines()                                 
  bbox = {}

       #  <frame>, <id>, <truncated>, <occluded>, <alpha>, <bb_left>, <bb_top>, <bb_right>, <bb_bottom>, <3D_H>, <3D_W>, <3D_L>, <x>,   <y>,   <z>,   <ry>
  mask = [False,   True,  False,       False,      False,   True,      True,     True,       True,        False,  False,  False,  False, False, False, False]
  
  for line in lines:
    l = line.strip().split(' ') #convert line to list
    typ = l.pop(2)  # get type of bbox 
    line = np.asarray(l).astype(np.float32) # convert into array 
    frame, line = int(line[0]), line[mask] # get frame number and mask the line   
    if frame not in bbox.keys():
      bbox[frame] = []   
    if typ in used_type:
        bbox[frame].append(line)
  return bbox

def add_bbox(img, bbox, color=(255, 0, 0), thickness=2):
  ''' 
    annotate an image with bounding boxes:
    supports single bbox or list of bboxs
  '''

  annotated = np.copy(img)
  if bbox: 
    if isinstance(bbox[0], np.ndarray) or isinstance(bbox[0], list):
        for (_,x1,y1,x2,y2) in bbox:
            cv.rectangle(annotated, (x1, y1), (x2, y2), color , thickness)
    else:
        _,x1,y1,x2,y2 = bbox
        cv.rectangle(annotated, (x1, y1), (x2, y2), color , thickness)
  
  return annotated

In [49]:
"""
r1: tuple (x1,y1,x2,y2) coordinates of rectangle 1
r2: tuple (x1,y1,x2,y2) coordinates of rectangle 2
"""
def intersection(r1, r2):
  if r1[0] >= r2[2] or r1[2] <= r2[0] or r1[1] >= r2[3] or r1[3] <= r2[1]:
    return False
  else:
    return True


##**Dataset**

https://iq.opengenus.org/object-detection-with-histogram-of-oriented-gradients-hog/

###Positive Dataset

Use provided bounding boxes from train dataset

###Negative Dataset

Use different objects from random patches in train dataset

In [61]:
from matplotlib.cbook import maxdict
import os
import cv2 as cv
from random import randint

positive_dataset = list()

negative_dataset = list()

number_of_negative_samples_per_frame = 10

minimum_bbox_h, maximum_bbox_h = 40, 150
minimum_bbox_w, maximum_bbox_w = 50, 200

max_diff_width_height = 20

for image_sequence in range(4):
  bboxes = parse('dataset/000{image_sequence}.txt'.format(image_sequence=image_sequence))
  for frame_id, frame_bboxes in bboxes.items():

    img = cv.imread('dataset/000{image_sequence}/{frame_id:06d}.png'.format(image_sequence=image_sequence, frame_id=frame_id))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

    for sample in range(number_of_negative_samples_per_frame):

        while(True):
          random_y1 = randint(0,img.shape[0])
          random_y2 = randint(random_y1,img.shape[0])

          random_x1 = randint(0,img.shape[1])
          random_x2 = randint(random_x1,img.shape[1])

          # validate dimensions
          if random_y2 - random_y1 < minimum_bbox_h or random_y2 - random_y1 > maximum_bbox_h:
            continue
          if random_x2 - random_x1 < minimum_bbox_h or random_x2 - random_x1 > maximum_bbox_w:
            continue

          if abs((random_x2 - random_x1) - (random_y2 - random_y1)) > max_diff_width_height:
            continue

          random_bbox = (random_x1, random_y1, random_x2, random_y2)

          # validate no-intersection with vehicles
          valid = True
          for (id, x1,y1,x2,y2) in frame_bboxes:
            if intersection((x1,y1,x2,y2), random_bbox):
              valid = False
              break

          if valid:
            break

        # extract patch
        random_bbox_img = img[random_y1:random_y2,random_x1:random_x2]
        negative_dataset.append(random_bbox_img)

    for bbox in frame_bboxes:
      id, x1, y1, x2, y2 = bbox.astype('int32')
      
      bbox_img = img[y1:y2,x1:x2]
      positive_dataset.append(bbox_img)